# CLASE 2.6: Árboles de decisión.
---
## Introducción.
En esta sección introduciremos un nuevo tipo de algoritmo de aprendizaje supervisado en la forma de un marco de referencia o *framework* unificado, conocido como **árbol de decisión**. Dicho *framework* se separa aún más de la teoría que hemos desarrollado previamente en el contexto de este tipo de modelos, debido a que, a diferencia de las máquinas de soporte vectorial que estudiamos en la clase 2.5, los árboles de decisión serán el primer tipo de modelo inherentemente no lineal que vamos a desarrollar. La razón fundamental de su estudio separado (y dedicado) estriba en dos pilares fundamentales: Su **simplicidad** y su **éxito** en la resolución de problemas tanto de clasificación como de regresión. Además, constituyen la base de muchos otros modelos que también veremos más adelante. Un ejemplo clásico es el **modelo de bosque aleatorizado** (del inglés **random forest**), que por supuesto estudiaremos en detalle en la clase 2.7.

El **éxito** de este tipo de modelos y, por extensión, de cualquier otro modelo derivado, guarda relación con varios factores esenciales:

- **No son modelos paramétricos**. Pueden, por tanto, modelar relaciones de alta complejidad entre variables de entrada y salida arbitrarias, **sin necesidad de ningún conocimiento específico a priori**.
- **Pueden tratar con conjuntos de datos heterogéneos**, ya sea compuestos de variables numéricas, discretas, categóricas, o un *mix* de todas ellas.
- **Implementan de forma intrínseca un procedimiento de selección de atributos**, haciéndolos una opción muy robusta para la eliminación de ruido generado por variables irrelevantes y/o redundantes (al menos, hasta cierto punto).
- **Son excelentes para conjuntos de entrenamiento con *outliers* o errores en las etiquetas asociadas a cada observación**.
- **Son fácilmente interpretables**, incluso para desarrolladores sin un *background* completo en aspectos estadísticos.

## Modelos con estructura de tipo árbol.
Sea $\mathcal{D} =\left\{ \left( \mathbf{X} ,\mathbf{y} \right)  :\mathbf{X} \in \mathbb{R}^{m\times n} \wedge \mathbf{y} \in \mathbb{R}^{m} \right\}$ un conjunto de entrenamiento arbitrario. Cuando $\mathbf{y}$ es un vector de valores discretos, como ocurre en un problema de clasificación (de tipo multinomial), su recorrido será finito y estará constituido por todas las $k$ clases o categorías posibles asociadas a las instancias de $\mathcal{D}$, digamos $\mathcal{Y} =\left\{ c_{1},\cdots ,c_{k}\right\}$. Una forma útil de visualizar un problema de este tipo es pensar que $\mathbf{y}$ define una partición sobre un universo de puntos, que llamamos $\Omega$, tal que

$$\Omega =\Omega_{c_{1}} \cup \Omega_{c_{2}} \cup \cdots \cup \Omega_{c_{k}} =\bigcup^{c_{k}}_{s=c_{1}} \Omega_{c_{s}}$$
<p style="text-align: right;">$(6.1)$</p>

donde $\Omega_{c_{s}}$ es el conjunto de instancias de $\mathcal{D}$ para las cuales $y_{i}$ tiene un valor igual a $c_{s}$ (siendo $y_{i}$ la clase asociada a la $i$-ésima instancia de $\mathcal{D}$). Similarmente, si $f$ es una hipótesis que hace el trabajo de *predecir* la clase asociada a una instancia $\mathbf{x}_{i}\in \mathbb{R}^{n}$ (donde $1\leq i\leq m$), entonces $f$ puede verse como una partición de $\Omega$, ya que ésta permite obtener una *estimación* $\hat{y}_{i}$ de $y_{i}$ para todo $i=1,...,m$. Sin embargo, dicha partición se define en el espacio de entrada $\mathcal{X}$ (que corresponde al dominio de la matriz $\mathbf{X}$) en vez de hacerlo en $\Omega$, lo que podemos escribir como

$$\mathcal{X} =\mathcal{X}^{f}_{c_{1}} \cup \mathcal{X}^{f}_{c_{2}} \cup \cdots \cup \mathcal{X}^{f}_{c_{k}} =\bigcup^{c_{k}}_{s=c_{1}} \mathcal{X}^{f}_{c_{s}}$$
<p style="text-align: right;">$(6.2)$</p>

donde $\mathcal{X}^{f}_{c_{s}}$ es un subconjunto que contiene a los vectores $\mathbf{x}_{i}\in \mathcal{X}$ tales que $f(\mathbf{x}_{i})=c_{s}$. Correspondientemente, el aprendizaje de un *predictor* $f$ puede reinterpretarse como una partición de $\mathcal{X}$ de manera que ésta coincida *lo mejor posible* con la partición original generada por $\mathbf{y}$, que –a su vez– es la *mejor partición posible* (o partición *óptima*).

Desde un punto de vista geométrico, el principio básico de los modelos de árbol de decisión es extraordinariamente simple. Consiste en aproximarse a la partición óptima mediante una secuencia de particiones de $\mathcal{D}$ en subespacios vectoriales, a partir de los cuales se asignan valores constantes como imágenes del predictor $f$ a todas las instancias que viven en cada uno de esos subespacios.

A fin de ir dando forma de manera más rigurosa a estos conceptos, vamos a formalizar algunas definiciones.

**<font color='blue'>Definición 6.1 – Árbol:</font>** Sea $G=(V,E)$ una red o grafo, donde $V$ es el conjunto de vértices o *nodos* y $E$ es el conjunto de arcos o *caminos*. Asumamos la siguiente terminología: Si $G$ contiene $p$ nodos, entonces $V$ puede expresarse por medio del conjunto $V=\left\{ 1,...,p\right\}$, siendo entonces $E=\left\{ \left( s,t\right)  :s\wedge t\in V,s\neq t\right\}$ el conjunto de todos los arcos de la red que unen a los nodos $s$ y $t$. Un **árbol** se define como un grafo $G=(V,E)$ tal que cualquier combinación de nodos está conectada por uno y sólo un arco.

Al respecto, es posible observar varios aspectos importantes en una red de tipo árbol:

- **(T1):** Si $G=(V,E)$ representa a un árbol, entonces es común designar a uno de sus nodos como la **raíz** del mismo. De ser así, entonces necesariamente $G$ será un **grafo dirigido** (es decir, los arcos $E$ tendrán **direcciones restringidas** –por ejemplo, $(s,t)$ podría recorrerse desde $i$ hasta $j$, pero no al revés–) para el cual todos los arcos se *alejarán* de dicho nodo raíz. Un árbol así definido es llamado **árbol con raíz**.
- **(T2):** Si existe un arco que va desde $s$ a $t$ (es decir, $(s,t)\in E$), entonces diremos que $s$ es el **nodo padre** (o, simplemente, *padre*) del nodo $j$. Correspondientemente, el nodo $j$ será llamado **nodo hijo** (o simplemente *hijo*) del nodo $i$.
- **(T3):** Para un árbol con raíz, diremos que un nodo es **interno** si tiene uno o más hijos, y **términal** si no los tiene. Los nodos terminales, sobretodo en la teoría de aprendizaje automatizado, suelen ser llamados **nodos hoja**, o simplemente **hojas**.
- **(T4):** Un **árbol binario** es un árbol con raíz para el cual cada nodo interno tiene siempre dos hijos.

En estos términos, un **modelo con estructura de tipo árbol** o, derechamente, **árbol de decisión**, puede definirse como un objeto matemático $f:\mathcal{X}\longrightarrow \mathcal{Y}$, representado por un árbol con raíz (con frecuencia, binario, aunque no necesariamente tiene que ser así), donde cualquier nodo $t$ representa un subespacio $\mathcal{X}_{t}\subset \mathcal{X}$ del espacio de entrada, donde el nodo raíz $t_{0}$ corresponde a todo el conjunto $\mathcal{X}$. Los nodos internos del árbol, digamos $t$, se etiquetan con una **separación** o **split** $s_{t}$ que se construye a partir de una serie de *"preguntas"* que forman un conjunto $\Omega$. Este *split* divide al subespacio $\mathcal{X}_{t}$ que representa al nodo $t$ en un número de subespacios disjuntos, cada uno de los cuales se corresponde con los nodos hijos de $t$. Por ejemplo, el conjunto de todos los *splits* binarios equivale a un conjunto $\Omega$ de preguntas $s$ de la forma: *"¿La instancia $\mathbf{x}_{i}$ pertence a $\mathcal{X}_{r}$?"*, donde $\mathcal{X}_{r}\subset \mathcal{X}$ es *algún* subconjunto del espacio de entrada. De esta manera, cualquier *split* $s$ divide a $\mathcal{X}_{t}$ en dos subespacios respectivamente. Si desginamos por $t$ a este nodo, entonces tales subespacios serán $\mathcal{X}_{t}\cap \mathcal{X}_{r}$ para el nodo hijo hacia la *izquierda* de $t$, y $\mathcal{X}_{t} \cap (\mathcal{X}\setminus  \mathcal{X}_{r})$ para el nodo hijo hacia la *derecha* de $t$. Los nodos terminales se etiquetan con las *mejores* estimaciones $\hat{y}_{t}\in \mathcal{Y}$ para la variable de salida u objetivo. Si $f$ representa a un **árbol de clasificación**, entonces $\hat{y}_{t} \in \left\{ c_{1},\cdots ,c_{k}\right\}$, mientras que, si $f$ es un **árbol de regresión**, entonces $\hat{y}\in [a,b]$, donde $[a,b]$ es un intervalo cerrado de $\mathbb{R}$. Bajo estas condiciones, la predicción $f(\mathbf{x}_{i})$ es el valor con el cual se etiqueta la hoja a la cual se llega mediante la instancia $\mathbf{x}_{i}$, al propagarse por el árbol siguiendo los *splits* $s_{t}$, lo que puede estructurarse conforme el algoritmo (6.1). Notemos que dicho algoritmo se escribe en *pseudo-código*.

**`def`** `predict`$(f,\mathbf{x}_{i})$<br>
>$t=t_{0}$<br>
**`while`** $t$ no sea un nodo terminal:<br>
>>$t=$el nodo hijo $t^{\ast}$ de $t$ tal que $\mathbf{x}_{i}\in \mathcal{X}_{t^{\ast}}$<br>

>**`return`** $\hat{y}_{t}$
<p style="text-align: center;">Algoritmo (6.1): Ejemplo básico de algoritmo que predice la salida $\hat{y}=f(\mathbf{x}_{i})$ en un árbol de decisión para una instacia $\mathbf{x}_{i}$</p>

La Fig. (6.1) ilustra un modelo de árbol de decisión $f$ compuesto por cinco nodos y que particiona el espacio de entrada $\mathcal{X}=\mathcal{X}_{1}\times \mathcal{X}_{2}=[0,1]\times [0,1]$ para un problema de clasificación binaria (donde el espacio de salida es $\mathcal{Y}=\left\{ c_{1},c_{2}\right\}$). El nodo $t_{0}$ es la raíz del árbol y corresponde al espacio de entrada completo; es decir, $\mathcal{X}_{t_{0}}=\mathcal{X}$. Dicho nodo se etiqueta con el *split* binario $s_{0}=\left\{ x^{\left( 0\right)  }_{1}\leq 0.7\right\}$ (es decir, dicho *split* formula la siguiente pregunta: *"¿Es $x$ menor o igual que $0.7$?"*), donde la notación se ha modificado convenientemente (y de manera un tanto redundante, pero solo por propósitos didácticos) para asignar a la instancia *propagada* en el árbol un índice asociado al nodo y la profundidad o *nivel* en el que nos encontramos en el árbol (en otras palabras, $x_{1}^{(0)}$ hace referencia al *split* para una instancia $x$, asociado al nodo $t_{0}$ y que define al nivel 1 del árbol de decisión –un nivel más abajo del nodo raíz–). El *split* $s_{0}$ divide al espacio $\mathcal{X}_{t_{0}}$ en dos subespacios disjuntos, $\mathcal{X}_{t_{1}}$ y $\mathcal{X}_{t_{2}}$, donde el primer subespacio corresponde al nodo hijo $t_{1}$ ubicado a la izquierda de $t_{0}$, y que representa al conjunto de todas las instancias $\mathbf{x}\in \mathcal{X}_{t_{0}}$ tales que $x\leq 0.7$.

<p style="text-align: center;"><img src="figures/fig_6_1.png" width="350"></p>
<p style="text-align: center;">Fig. (6.1): Un sencillo árbol de decisión construido para resolver un problema de clasificación a partir de<br>un espacio de entrada definido por el conjunto $\mathcal{X}=[0,1]\times [0,1]$</p>